In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [2]:
data=open('data/gitanjali.txt','r').read()
data=data.decode('ascii','ignore').encode('ascii')
data[:100]

'Thou hast made me endless, such is thy pleasure.  This frail\r\nvessel thou emptiest again and again, '

In [3]:
#create_vocab

vocab=list(set(list(data)))
data=list(data)
vocab_len=len(vocab)
print("Vocabulary :",vocab,"Vocab Size :",vocab_len)

char_to_one_hot={}
for i in range(vocab_len):
    x=np.zeros(vocab_len)
    x[i]=1.
    char_to_one_hot[vocab[i]]=x
    


('Vocabulary :', ['\n', '\r', '!', ' ', "'", '-', ',', '.', ';', '?', 'A', 'C', 'B', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'J', 'M', 'L', 'O', 'N', 'P', 'S', 'R', 'U', 'T', 'W', 'V', 'Y', '_', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'], 'Vocab Size :', 60)


In [15]:
# config
win_size=4

n_input=vocab_len
n_hidden=128
n_timesteps=win_size
n_classes=vocab_len

learning_rate=0.01
epochs=50
batch_size=1
display_step=200

def next_batch(idx,batch_size=200,win_size=4):
    char_arr=data[idx:idx+win_size]
    training_data=np.zeros((win_size,vocab_len))
    for i in range(len(char_arr)):
        training_data[i,:]=char_to_one_hot[char_arr[i]]
    x=training_data
    y=char_to_one_hot[data[idx+1]]
    return x,y


In [5]:
#define model

X=tf.placeholder(tf.float32,[None,n_timesteps,n_input])
Y=tf.placeholder(tf.float32,[None,n_classes])

weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def RNN(x, weights, biases):
    x = tf.unstack(x, n_timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits=RNN(X,weights,biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for i in range(epochs):
        for j in range(len(data)-1):
            batch_x, batch_y = next_point(j)
            batch_x=np.reshape(batch_x,(1,batch_x.shape[0],batch_x.shape[1]))
            batch_y=np.reshape(batch_y,(1,batch_y.shape[0]))
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            if j % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                print("Step " + str(j) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

print("Optimization Finished!")

Step 0, Minibatch Loss= 3.7082, Training Accuracy= 0.000
Step 200, Minibatch Loss= 3.6903, Training Accuracy= 0.000
Step 400, Minibatch Loss= 1.5632, Training Accuracy= 1.000
Step 600, Minibatch Loss= 1.2953, Training Accuracy= 1.000
Step 800, Minibatch Loss= 1.4032, Training Accuracy= 1.000
Step 1000, Minibatch Loss= 1.2117, Training Accuracy= 1.000
Step 1200, Minibatch Loss= 0.4262, Training Accuracy= 1.000
Step 1400, Minibatch Loss= 1.2467, Training Accuracy= 1.000
Step 1600, Minibatch Loss= 3.5289, Training Accuracy= 0.000
Step 1800, Minibatch Loss= 0.8384, Training Accuracy= 1.000
Step 2000, Minibatch Loss= 0.7909, Training Accuracy= 1.000
Step 2200, Minibatch Loss= 4.8058, Training Accuracy= 0.000
Step 2400, Minibatch Loss= 1.0448, Training Accuracy= 1.000
Step 2600, Minibatch Loss= 0.4078, Training Accuracy= 1.000
Step 2800, Minibatch Loss= 0.7738, Training Accuracy= 1.000
Step 3000, Minibatch Loss= 0.3157, Training Accuracy= 1.000
Step 3200, Minibatch Loss= 0.2282, Training Acc

KeyboardInterrupt: 

64013